<a href="https://colab.research.google.com/github/HazemmoAlsady/AWN_Graduation_Project/blob/main/Hazem's%20edits/Recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
import pickle

BASE_PATH = "/content/drive/MyDrive/graduation_project/final_model"

with open(f"{BASE_PATH}/need_level_pipeline.pkl", "rb") as f:
    need_pipeline = pickle.load(f)

with open(f"{BASE_PATH}/assistance_type_pipeline.pkl", "rb") as f:
    assist_pipeline = pickle.load(f)

with open(f"{BASE_PATH}/need_level_encoder.pkl", "rb") as f:
    need_encoder = pickle.load(f)

with open(f"{BASE_PATH}/assistance_type_encoder.pkl", "rb") as f:
    assist_encoder = pickle.load(f)


In [25]:
print("Need Level:", need_encoder.classes_)
print("Assist Type:", assist_encoder.classes_)


Need Level: ['High' 'Low' 'Medium']
Assist Type: ['disability_equipment' 'education' 'financial' 'food' 'housing' 'medical']


In [26]:
orgs = pd.DataFrame([
    {
        "org": "جمعية الأورمان",
        "assistance_types": ["food", "financial", "housing"],
        "cities": ["القاهرة", "الجيزة"],
        "priority": 5
    },
    {
        "org": "جمعية رسالة",
        "assistance_types": ["education", "medical"],
        "cities": ["القاهرة", "الإسكندرية"],
        "priority": 4
    },
    {
        "org": "بنك الطعام",
        "assistance_types": ["food"],
        "cities": ["القاهرة", "الجيزة"],
        "priority": 5
    },
    {
        "org": "جمعية التكافل الاجتماعي",
        "assistance_types": ["financial", "housing"],
        "cities": ["سوهاج", "أسيوط"],
        "priority": 3
    },
    {
        "org": "مؤسسة مجدي يعقوب",
        "assistance_types": ["medical"],
        "cities": ["أسوان"],
        "priority": 5
    },
    {
        "org": "مؤسسة مصر الخير",
        "assistance_types": ["food", "education", "financial", "housing"],
        "cities": ["القاهرة", "الجيزة"],
        "priority": 5
    }
])


In [27]:
def recommend_organizations(
    assistance_type,
    city,
    need_level,
    orgs_df,
    top_n=3
):
    scores = []

    for _, row in orgs_df.iterrows():
        score = 0

        # Assistance match
        if assistance_type in row["assistance_types"]:
            score += 4

        # City match
        if city in row["cities"]:
            score += 3

        # Need level weight
        if need_level == "High":
            score += 2
        elif need_level == "Medium":
            score += 1

        # Org priority
        score += row["priority"]

        scores.append(score)

    ranked = orgs_df.copy()
    ranked["score"] = scores

    return ranked.sort_values("score", ascending=False).head(top_n)


In [28]:
def choose_final_organization(recommendations_df):
    return (
        recommendations_df
        .sort_values("score", ascending=False)
        .iloc[0]["org"]
    )


In [29]:
def full_ai_pipeline(input_df):
    # Need Level
    need_pred = need_pipeline.predict(input_df)[0]
    need_level = need_encoder.inverse_transform([need_pred])[0]

    # Assistance Type
    assist_pred = assist_pipeline.predict(input_df)[0]
    assistance_type = assist_encoder.inverse_transform([assist_pred])[0]

    # Recommendation
    ranked_orgs = recommend_organizations(
        assistance_type=assistance_type,
        city=input_df.iloc[0]["city"],
        need_level=need_level,
        orgs_df=orgs,
        top_n=3
    )

    final_org = choose_final_organization(ranked_orgs)

    return {
        "predicted_need_level": need_level,
        "predicted_assistance_type": assistance_type,
        "final_recommended_org": final_org,
        "top_organizations": ranked_orgs[["org", "score"]]
    }


In [30]:
sample_case = pd.DataFrame([{
    "family_size": 5,
    "income_monthly": 1500,
    "monthly_expenses": 3500,
    "debts": 9000,
    "number_of_children": 3,
    "age": 40,
    "expense_to_income_ratio": 2.3,
    "case_type": "أرملة",
    "housing_type": "إيجار",
    "health_status": "مزمن",
    "city": "القاهرة",
    "gender": "female",
    "request_text_clean": "تحتاج مساعدة عاجلة في السكن"
}])

result = full_ai_pipeline(sample_case)
result


{'predicted_need_level': 'Medium',
 'predicted_assistance_type': 'financial',
 'final_recommended_org': 'جمعية الأورمان',
 'top_organizations':                org  score
 0   جمعية الأورمان     13
 5  مؤسسة مصر الخير     13
 2       بنك الطعام      9}